In [6]:
import sys
sys.path.append("/root/GanLuo/PullSum_MNIST/CIFAR10实验_arxiv")
import torch
import numpy as np
from resnet_model import new_ResNet18
from prepare_data import distributed_cifar10_dataloaders, hetero_distributed_cifar10_dataloaders
from training import train_PullSum, train_PullDiag, train_FRSD, train_FROZEN
from useful_functions import *
from new_network_func import *

In [7]:
n = 20
M = 5
batch_size = 128*M
p=0.5

I = np.eye(n)
one = np.ones(n)
R = np.outer(one, one) / n
from network_func import di_ring
A,C=di_ring(n=n)
A_M = np.linalg.matrix_power(A, M)
n = A.shape[0]
one = np.ones(n)
result = np.dot(one,A_M)
t = np.diag(1/result)
A=A_M
B=A_M@t
CIFAR10_root = (
    "/root/GanLuo/PullSum_MNIST/CIFAR10实验_arxiv/cifar-10-python/cifar-10-batches-py"
)
trainloader_list, testloader = hetero_distributed_cifar10_dataloaders(
    n, batch_size=batch_size, root=CIFAR10_root, seed=42,alpha=p
)

In [ ]:
lr = 1e-3 
beta = 0.1
epochs = 400
warm_up = False

train_loss_history, test_loss_history, test_accuracy_history = train_PullSum(
    n=n,
    A=A,
    B=B,
    model_class=new_ResNet18,
    seed_for_model=42,
    epochs=epochs,
    lr=lr,
    trainloader_list=trainloader_list,
    testloader=testloader,
    show_graph=True,
    batch_size=batch_size,
    csv_root=f"/root/GanLuo/PullSum_MNIST/CIFAR10实验_arxiv/数值实验_multi_gossip/Ring1, p={p}, n={n}, lr={lr}, M={M}, batch_size={batch_size}.csv",
    warm_up=warm_up,
)

optimizer初始化成功!


Training Progress:   2%|▎         | 10/400 [01:49<1:04:31,  9.93s/it, epoch=10, test_accuracy=10.0000000000%, test_loss=2.3057608223, train_loss=2.3253388822]